# Downloade Twitter-usernamen aller Unions Abgeordneten

__Nach Anleitung (https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html)__

__Fraktionsseite: https://www.cducsu.de/hier-stellt-die-cducsu-bundestagsfraktion-ihre-abgeordneten-vor__

In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('Union_twitter.json', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess
import logging



class CDUSpider(scrapy.Spider):
    name = "CDU_Abgeordnete_Twitter"
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'Union_twitter.json'                      # Used for pipeline 2
    }

    def start_requests(self):
        urls = [
            'https://www.cducsu.de/hier-stellt-die-cducsu-bundestagsfraktion-ihre-abgeordneten-vor',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        for delegate in response.css('div.delegates-list'):
            yield {
                #'text': delegate.css('li.twitter::text').getall(),
                'name': delegate.xpath('//a[contains(@href, "twitter.com")]/@href').getall(),
            }


In [4]:
process3 = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process3.crawl(CDUSpider)
process3.start()

2020-04-06 15:39:17 [scrapy.utils.log] INFO: Scrapy 1.7.3 started (bot: scrapybot)
2020-04-06 15:39:17 [scrapy.utils.log] INFO: Versions: lxml 4.2.1.0, libxml2 2.9.4, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.7.0, Python 3.6.9 (default, Nov  7 2019, 10:44:02) - [GCC 8.3.0], pyOpenSSL 19.0.0 (OpenSSL 1.1.1  11 Sep 2018), cryptography 2.1.4, Platform Linux-4.15.0-91-generic-x86_64-with-Ubuntu-18.04-bionic
2020-04-06 15:39:17 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'Union_twitter.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [5]:
# Kürze Twitter URL mit Regulären Ausdrücken sodass nur noch der Username übrig bleibt

import json
import re

with open("Union_twitter.json", "r") as Union:
    data = json.load(Union)

twitter_names = []
for twitter_url in data['name']:
    #print(twitter_url)
    twitter_names.append(re.sub("http(.|):\/\/(www.|)twitter.com\/","",twitter_url))

with open("Union_twitter_clean.json", "w") as write_file:
     json.dump(twitter_names, write_file)
    

JSONDecodeError: Extra data: line 2 column 1 (char 4875)